# 📋 Documentation Complète du Système d'Authentification MSPR

## 🎯 Objectif du Projet
Ce notebook documente intégralement le **système d'authentification sécurisé MSPR** développé dans le cadre du projet TPRE912.

### 🔧 Technologies Utilisées
- **Backend** : Flask (Python)
- **Base de données** : PostgreSQL (Docker)
- **Authentification** : 2FA avec TOTP (Google Authenticator)
- **Chiffrement** : Fernet (cryptography)
- **QR Codes** : Génération automatique pour mots de passe et 2FA

### 📚 Structure du Projet
Le système comprend plusieurs handlers spécialisés :
- **`handler.py`** : Création d'utilisateurs
- **`login_handler.py`** : Authentification et vérification d'expiration
- **`generate_2fa_handler.py`** : Régénération des codes 2FA
- **Interface web** : Templates Flask pour l'interaction utilisateur

---

## 1. 📦 Importation des Bibliothèques Nécessaires

Le système utilise plusieurs bibliothèques Python spécialisées pour assurer la sécurité et les fonctionnalités d'authentification :

In [ ]:
# Importations du système MSPR
import json          # Traitement des données JSON (requêtes/réponses)
import random        # Génération aléatoire pour les mots de passe
import string        # Constantes pour les caractères utilisables
import psycopg2      # Connecteur PostgreSQL
import qrcode        # Génération de QR codes
import base64        # Encodage des images en base64
import io            # Gestion des flux mémoire
import os            # Variables d'environnement (non utilisées ici)
import pyotp         # Génération et vérification TOTP (2FA)
from datetime import datetime    # Gestion des dates
from cryptography.fernet import Fernet  # Chiffrement symétrique

print("✅ Toutes les bibliothèques sont importées avec succès !")

## 2. 🔐 Fonction `get_encryption_key()` - Clé de Chiffrement

Cette fonction est **cruciale** pour la sécurité du système. Elle retourne une clé de chiffrement fixe utilisée pour :
- Chiffrer les mots de passe en base de données
- Chiffrer les secrets 2FA
- Garantir la cohérence entre tous les handlers

⚠️ **Important** : La même clé doit être utilisée dans tous les handlers pour pouvoir déchiffrer les données.

In [ ]:
def get_encryption_key():
    """
    Clé de chiffrement fixe pour le projet MSPR
    Retourne une clé Fernet valide pour le chiffrement symétrique
    """
    # Clé fixe pour le projet - MÊME CLÉ POUR TOUS LES HANDLERS !
    return b'ZmDfcTF7_60GrrY167zsiPd67pEvs0aGOv2oasOM1Pg='

# Test de la fonction
key = get_encryption_key()
print(f"🔑 Clé de chiffrement : {key}")
print(f"📏 Longueur de la clé : {len(key)} bytes")
print(f"✅ Clé valide pour Fernet : {len(key) == 44 and key.endswith(b'=')}")

## 3. 🛡️ Initialisation de Fernet - Moteur de Chiffrement

Fernet est un système de chiffrement symétrique qui garantit :
- **Confidentialité** : Les données sont illisibles sans la clé
- **Intégrité** : Détection des modifications non autorisées
- **Authentification** : Vérification de l'origine des données

In [ ]:
# Initialisation du moteur de chiffrement
FERNET_KEY = get_encryption_key()
fernet = Fernet(FERNET_KEY)

# Test du chiffrement/déchiffrement
test_data = "Mot de passe secret"
encrypted = fernet.encrypt(test_data.encode())
decrypted = fernet.decrypt(encrypted).decode()

print(f"🔤 Données originales : {test_data}")
print(f"🔒 Données chiffrées : {encrypted}")
print(f"🔓 Données déchiffrées : {decrypted}")
print(f"✅ Test de chiffrement : {'SUCCÈS' if test_data == decrypted else 'ÉCHEC'}")

## 4. 🗄️ Configuration de la Base de Données PostgreSQL

Le système utilise PostgreSQL en conteneur Docker avec une configuration fixe pour simplifier le déploiement éducatif.

In [ ]:
# Configuration base de données - VALEURS FIXES POUR LE PROJET
DB_HOST = 'localhost'        # PostgreSQL Docker sur localhost
DB_NAME = 'cofrap'          # Nom de la base de données
DB_USER = 'postgres'        # Utilisateur PostgreSQL
DB_PASSWORD = 'mspr2024'    # Mot de passe PostgreSQL Docker

print("🐘 Configuration PostgreSQL :")
print(f"   🏠 Hôte : {DB_HOST}")
print(f"   🗃️  Base : {DB_NAME}")
print(f"   👤 Utilisateur : {DB_USER}")
print(f"   🔒 Mot de passe : {'*' * len(DB_PASSWORD)}")

# Structure de la table users
print("\n📋 Structure de la table 'users' :")
table_structure = """
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    password TEXT NOT NULL,           -- Mot de passe chiffré
    secret_2fa TEXT NOT NULL,         -- Secret TOTP chiffré
    gendate TIMESTAMP NOT NULL,       -- Date de création
    expired BOOLEAN DEFAULT FALSE     -- Statut d'expiration
);
"""
print(table_structure)

## 5. 🎲 Fonction `generate_password()` - Génération de Mots de Passe Sécurisés

Cette fonction génère des mots de passe robustes avec :
- **Longueur** : 24 caractères par défaut
- **Caractères sûrs** : Lettres, chiffres, symboles spéciaux
- **Aléatoire cryptographique** : Utilisation de `SystemRandom`

In [ ]:
def generate_password(length=24):
    """
    Génère un mot de passe sécurisé avec des caractères ASCII sûrs
    """
    # Utiliser seulement des caractères ASCII sûrs (évite les problèmes d'encodage)
    chars = string.ascii_letters + string.digits + "!@#$%^&*"
    return ''.join(random.SystemRandom().choice(chars) for _ in range(length))

# Tests de génération de mots de passe
print("🔐 Exemples de mots de passe générés :")
for i in range(3):
    password = generate_password()
    print(f"   {i+1}. {password} (longueur: {len(password)})")

# Test avec différentes longueurs
print("\n📏 Mots de passe de différentes longueurs :")
for length in [8, 12, 16, 24]:
    password = generate_password(length)
    print(f"   {length} chars: {password}")

## 6. 📱 Fonction `generate_qrcode()` - Génération de QR Codes

Cette fonction transforme n'importe quelle donnée textuelle en QR code image encodée en base64 pour l'affichage web.

In [ ]:
def generate_qrcode(data):
    """
    Génère un QR code à partir de données textuelles
    Retourne l'image encodée en base64 pour l'affichage web
    """
    qr = qrcode.make(data)
    buffered = io.BytesIO()
    qr.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Test de génération de QR code
test_username = "jean.dupont"
test_password = generate_password(12)
qr_data = f"Username: {test_username}\nPassword: {test_password}"

qr_code_b64 = generate_qrcode(qr_data)

print("📱 Génération de QR Code :")
print(f"   📝 Données : {qr_data}")
print(f"   📏 Taille base64 : {len(qr_code_b64)} caractères")
print(f"   🖼️  Début de l'image : {qr_code_b64[:50]}...")
print("   ✅ QR Code généré avec succès !")

## 7. 💾 Fonction `insert_user()` - Insertion en Base de Données

Cette fonction insère un nouvel utilisateur avec ses données chiffrées dans la base PostgreSQL.

In [ ]:
def insert_user(username, encrypted_password, encrypted_2fa_secret, gendate):
    """
    Insère un nouvel utilisateur dans la base de données PostgreSQL
    Tous les paramètres sensibles sont déjà chiffrés
    """
    conn = psycopg2.connect(
        host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD
    )
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO users (username, password, secret_2fa, gendate, expired)
        VALUES (%s, %s, %s, %s, %s)
    """, (username, encrypted_password, encrypted_2fa_secret, gendate, False))
    conn.commit()
    cur.close()
    conn.close()

# Simulation d'insertion (sans exécution réelle)
print("💾 Simulation d'insertion utilisateur :")
print("   📝 Paramètres requis :")
print("      - username : Nom d'utilisateur unique")
print("      - encrypted_password : Mot de passe chiffré avec Fernet")
print("      - encrypted_2fa_secret : Secret TOTP chiffré avec Fernet")
print("      - gendate : Date de création (datetime UTC)")
print("      - expired : False par défaut (nouveau compte)")
print("\n   🔄 Processus :")
print("      1. Connexion à PostgreSQL")
print("      2. Exécution de la requête INSERT")
print("      3. Validation (COMMIT)")
print("      4. Fermeture de la connexion")

## 8. 🔐 Configuration et Génération du Secret TOTP (2FA)

Le système utilise TOTP (Time-based One-Time Password) compatible avec Google Authenticator et autres applications 2FA.

In [ ]:
# Configuration 2FA
APP_NAME = "MSPR App"          # Nom de l'application
ISSUER_NAME = "MSPR Security"  # Nom de l'organisation

def generate_totp_secret():
    """
    Génère un secret TOTP aléatoire de 32 caractères base32
    Compatible avec Google Authenticator, Authy, etc.
    """
    return pyotp.random_base32()

# Test de génération de secrets TOTP
print("🔐 Configuration 2FA :")
print(f"   📱 Application : {APP_NAME}")
print(f"   🏢 Émetteur : {ISSUER_NAME}")

print("\n🎲 Génération de secrets TOTP :")
for i in range(3):
    secret = generate_totp_secret()
    print(f"   {i+1}. {secret} (longueur: {len(secret)})")

# Test de génération de codes avec un secret
test_secret = generate_totp_secret()
totp = pyotp.TOTP(test_secret)
current_code = totp.now()

print(f"\n⏰ Test de génération de code temporaire :")
print(f"   🔑 Secret : {test_secret}")
print(f"   🔢 Code actuel : {current_code}")
print(f"   ⏱️  Validité : 30 secondes")

## 9. 📱 Fonction `generate_totp_qr_code()` - QR Code pour 2FA

Cette fonction génère un QR code spécialement formaté pour les applications d'authentification 2FA.

In [ ]:
def generate_totp_qr_code(username, secret):
    """
    Génère un QR Code pour TOTP (Google Authenticator)
    Format URI standardisé pour les apps 2FA
    """
    totp_uri = pyotp.totp.TOTP(secret).provisioning_uri(
        name=username,
        issuer_name=ISSUER_NAME
    )
    
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(totp_uri)
    qr.make(fit=True)
    
    qr_image = qr.make_image(fill_color="black", back_color="white")
    buffered = io.BytesIO()
    qr_image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Test de génération de QR code 2FA
test_username = "jean.dupont"
test_secret = generate_totp_secret()
totp_uri = pyotp.totp.TOTP(test_secret).provisioning_uri(
    name=test_username,
    issuer_name=ISSUER_NAME
)

print("📱 Génération de QR Code 2FA :")
print(f"   👤 Utilisateur : {test_username}")
print(f"   🔑 Secret : {test_secret}")
print(f"   🔗 URI TOTP : {totp_uri}")

qr_2fa = generate_totp_qr_code(test_username, test_secret)
print(f"   📏 Taille QR : {len(qr_2fa)} caractères")
print("   ✅ QR Code 2FA généré avec succès !")

## 10. 🎯 Fonction `handle()` - Orchestration de la Création d'Utilisateur

La fonction `handle()` est le **point d'entrée principal** qui orchestre toute la création d'un utilisateur sécurisé.

In [ ]:
def handle(req):
    """
    Fonction principale d'orchestration pour créer un utilisateur
    
    Flux de traitement :
    1. Extraction du nom d'utilisateur depuis la requête JSON
    2. Génération d'un mot de passe sécurisé
    3. Chiffrement du mot de passe
    4. Génération d'un secret TOTP pour 2FA
    5. Chiffrement du secret 2FA
    6. Génération des QR codes (mot de passe + 2FA)
    7. Insertion en base de données
    8. Construction de la réponse JSON
    """
    try:
        # 1. Extraction des paramètres
        body = json.loads(req)
        username = body.get('username')

        if not username:
            return json.dumps({
                "error": "Username is required",
                "status": "error"
            })

        # 2. Génération du mot de passe
        password = generate_password()
        encrypted_password = fernet.encrypt(password.encode()).decode()

        # 3. Génération du secret 2FA
        totp_secret = generate_totp_secret()
        encrypted_2fa_secret = fernet.encrypt(totp_secret.encode()).decode()

        # 4. Génération des QR Codes
        password_qr = generate_qrcode(f"Username: {username}\\nPassword: {password}")
        totp_qr = generate_totp_qr_code(username, totp_secret)

        # 5. Insertion en base de données
        gendate = datetime.utcnow()
        insert_user(username, encrypted_password, encrypted_2fa_secret, gendate)

        # 6. Construction de la réponse
        response = {
            "message": "User created successfully",
            "username": username,
            "password": password,
            "password_qr_code": password_qr,
            "totp_secret": totp_secret,
            "totp_qr_code": totp_qr,
            "instructions": {
                "step1": "Save your password securely",
                "step2": "Scan the TOTP QR code with Google Authenticator",
                "step3": "Use both password and 2FA code to login"
            },
            "status": "success"
        }

        return json.dumps(response)

    except Exception as e:
        # Gestion des erreurs
        error_response = {
            "error": f"Unexpected error: {str(e)}",
            "status": "error"
        }
        return json.dumps(error_response)

print("🎯 Fonction handle() définie - Prête à orchestrer la création d'utilisateurs !")

## 11. 🔄 Schéma de Fonctionnement Global des Composants

### 📊 Architecture du Système MSPR

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   Frontend      │    │   Handlers      │    │   PostgreSQL    │
│   (Flask)       │    │   (Python)      │    │   (Docker)      │
│                 │    │                 │    │                 │
│ • Templates     │◄──►│ • handler.py    │◄──►│ • Base cofrap   │
│ • Forms         │    │ • login_handler │    │ • Table users   │
│ • QR Display    │    │ • generate_2fa  │    │ • Données       │
│ • Messages      │    │ • Validation    │    │   chiffrées     │
└─────────────────┘    └─────────────────┘    └─────────────────┘
         │                       │                       │
         └───────────────────────┼───────────────────────┘
                                 │
                    ┌─────────────────┐
                    │   Sécurité      │
                    │                 │
                    │ • Fernet        │
                    │ • TOTP/2FA      │
                    │ • QR Codes      │
                    │ • Expiration    │
                    └─────────────────┘
```

### 🚀 Flux de Création d'Utilisateur

1. **Réception** : `handle()` reçoit une requête JSON avec `username`
2. **Génération** : Mot de passe sécurisé (24 chars) + Secret TOTP
3. **Chiffrement** : Les données sensibles sont chiffrées avec Fernet
4. **QR Codes** : Génération des QR codes pour mot de passe et 2FA
5. **Persistance** : Insertion en base PostgreSQL
6. **Réponse** : JSON avec toutes les informations + instructions

### 🔐 Flux d'Authentification (login_handler.py)

1. **Vérification expiration** : Contrôle des 6 mois d'ancienneté
2. **Déchiffrement** : Récupération du mot de passe et secret 2FA
3. **Validation** : Vérification mot de passe + code TOTP
4. **Marquage** : Mise à jour du statut d'expiration si nécessaire

## 12. 🧪 Test Complet du Système

Simulation d'une création d'utilisateur complète pour valider le fonctionnement de tous les composants.

In [ ]:
# Test complet du système (simulation sans base de données)
import json

def simulate_user_creation(username):
    """
    Simule la création complète d'un utilisateur
    """
    print(f"🚀 === SIMULATION CRÉATION UTILISATEUR : {username} ===")
    
    # 1. Génération des éléments de sécurité
    password = generate_password()
    encrypted_password = fernet.encrypt(password.encode()).decode()
    
    totp_secret = generate_totp_secret()
    encrypted_2fa_secret = fernet.encrypt(totp_secret.encode()).decode()
    
    print(f"✅ Mot de passe généré : {password}")
    print(f"✅ Secret 2FA généré : {totp_secret}")
    
    # 2. Génération des QR Codes
    password_qr = generate_qrcode(f"Username: {username}\\nPassword: {password}")
    totp_qr = generate_totp_qr_code(username, totp_secret)
    
    print(f"✅ QR mot de passe : {len(password_qr)} caractères")
    print(f"✅ QR 2FA : {len(totp_qr)} caractères")
    
    # 3. Test de déchiffrement
    decrypted_password = fernet.decrypt(encrypted_password.encode()).decode()
    decrypted_2fa = fernet.decrypt(encrypted_2fa_secret.encode()).decode()
    
    print(f"✅ Test déchiffrement mot de passe : {'OK' if password == decrypted_password else 'ERREUR'}")
    print(f"✅ Test déchiffrement 2FA : {'OK' if totp_secret == decrypted_2fa else 'ERREUR'}")
    
    # 4. Test de génération de code TOTP
    totp = pyotp.TOTP(totp_secret)
    current_code = totp.now()
    print(f"✅ Code TOTP actuel : {current_code}")
    
    # 5. Construction de la réponse finale
    response = {
        "message": "User created successfully",
        "username": username,
        "password": password,
        "password_qr_code": password_qr,
        "totp_secret": totp_secret,
        "totp_qr_code": totp_qr,
        "instructions": {
            "step1": "Save your password securely",
            "step2": "Scan the TOTP QR code with Google Authenticator",
            "step3": "Use both password and 2FA code to login"
        },
        "status": "success"
    }
    
    print(f"✅ Réponse JSON générée : {len(json.dumps(response))} caractères")
    print("🎉 SIMULATION RÉUSSIE - Tous les composants fonctionnent !")
    
    return response

# Test avec un utilisateur exemple
test_user = simulate_user_creation("marie.martin")

## 13. 🛠️ Méthode de Mise en Place et Déploiement

### 🐳 Prérequis Docker
```bash
# 1. Lancer PostgreSQL en conteneur
docker run --name postgres-mspr -e POSTGRES_DB=cofrap -e POSTGRES_PASSWORD=mspr2024 -p 5432:5432 -d postgres:13

# 2. Créer la table users
docker exec -it postgres-mspr psql -U postgres -d cofrap -c "
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    password TEXT NOT NULL,
    secret_2fa TEXT NOT NULL,
    gendate TIMESTAMP NOT NULL,
    expired BOOLEAN DEFAULT FALSE
);"
```

### 📋 Scripts de Démarrage Automatisés
Le projet inclut des scripts centralisés dans `scripts-demarrage/` :

- **`DEMARRER-MSPR.bat`** : Lanceur principal
- **`start_app_docker.bat`** : Démarre PostgreSQL + Application
- **`start_app_powershell.ps1`** : Version PowerShell
- **`verifier-utilisateurs.bat`** : Contrôle des utilisateurs

### 🔧 Configuration Système
Toutes les variables sont **écrites en dur** dans le code :
- `DB_HOST = 'localhost'`
- `DB_NAME = 'cofrap'`
- `DB_USER = 'postgres'`
- `DB_PASSWORD = 'mspr2024'`
- `FERNET_KEY = 'ZmDfcTF7_60GrrY167zsiPd67pEvs0aGOv2oasOM1Pg='`

### 📱 Utilisation
1. **Création** : POST vers `/create-user` avec `{"username": "nom"}`
2. **Connexion** : POST vers `/login` avec `{"username": "nom", "password": "mdp", "totp_code": "123456"}`
3. **Interface** : Pages web Flask avec formulaires et QR codes

## 14. 📋 Résumé et Points Clés

### ✅ Fonctionnalités Implémentées

1. **Sécurité** :
   - Mots de passe chiffrés avec Fernet
   - Authentification 2FA avec TOTP
   - Expiration automatique des comptes (6 mois)
   - QR codes pour faciliter l'usage

2. **Architecture** :
   - Handlers spécialisés (création, connexion, 2FA)
   - Base PostgreSQL en conteneur Docker
   - Interface web Flask avec templates
   - Configuration centralisée et fixe

3. **Robustesse** :
   - Gestion d'erreurs complète
   - Validation des entrées
   - Messages utilisateur clairs
   - Tests de déploiement automatisés

### 🎯 Objectifs MSPR Atteints

- ✅ **Sécurité** : Chiffrement, 2FA, expiration
- ✅ **Usabilité** : Interface claire, QR codes, instructions
- ✅ **Déploiement** : Scripts automatisés, Docker
- ✅ **Documentation** : README complet, guides de dépannage
- ✅ **Maintenabilité** : Code structuré, handlers séparés

### 🚀 Prêt pour Production Éducative

Le système est **entièrement fonctionnel** pour un environnement éducatif avec :
- Configuration simplifiée (valeurs en dur)
- Déploiement Docker standardisé
- Documentation complète
- Tests de validation intégrés

---

**📖 Pour plus de détails, consultez le `README_NEW.md` complet du projet !**